<a href="https://colab.research.google.com/github/bvolcan/treinamento-h2ia/blob/main/Semana3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
import pandas as pd

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names

--2023-03-22 13:58:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 373704 (365K) [application/x-httpd-php]
Saving to: ‘agaricus-lepiota.data.1’

agaricus-lepiota.da 100%[===================>] 364.95K  --.-KB/s    in 0.05s   

2023-03-22 13:58:06 (7.00 MB/s) - ‘agaricus-lepiota.data.1’ saved [373704/373704]

--2023-03-22 13:58:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6816 (6.7K) [application/x-httpd-php]
Saving to: ‘agaricus-lepiota.names.1’

agaricus-lepiota.na 10

In [158]:
columns=[
  "class",
  "cap-shape",
  "cap-surface",
  "cap-color",
  "bruises",
  "odor",
  "gill-attachment",
  "gill-spacing",
  "gill-size",
  "gill-color",
  "stalk-shape",
  "stalk-root",
  "stalk-surface-above-ring",
  "stalk-surface-below-ring",
  "stalk-color-above-ring",
  "stalk-color-below-ring",
  "veil-type",
  "veil-color",
  "ring-number",
  "ring-type",
  "spore-print-color",
  "population",
  "habitat"
]
mushroom_raw_data = pd.read_csv("./agaricus-lepiota.data",names=columns)

In [159]:
mushroom_raw_data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [160]:
print(mushroom_raw_data["class"])

0       p
1       e
2       e
3       p
4       e
       ..
8119    e
8120    e
8121    e
8122    p
8123    e
Name: class, Length: 8124, dtype: object


In [161]:
###Teste de binarização utilizando OneHotEncoder

# from sklearn.preprocessing import OneHotEncoder

# oneHot = OneHotEncoder(handle_unknown='ignore')
# oneHot.fit(mushroom_raw_data)

# oneHot.get_feature_names_out()

# mushroom_processed_data = oneHot.transform(mushroom_raw_data)
# mushroom_processed_data = pd.DataFrame(mushroom_processed_data.toarray(),columns=oneHot.get_feature_names_out())
# mushroom_processed_data

In [343]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(mushroom_raw_data, test_size=0.3,stratify=mushroom_raw_data["class"])
print(len(train), len(test))

5686 2438


In [163]:
import numpy as np
import math

def find_entropy(data):
  entropy = 0
  for i in range(data.nunique()):
    x = data.value_counts()[i]/data.shape[0] 
    entropy += (- x * math.log(x,2))
  return round(entropy,3)

def information_gain(data, data_):
  info = 0
  for i in range(data_.nunique()):
    df = data[data_ == data_.unique()[i]]
    w_avg = df.shape[0]/data.shape[0]
    entropy = find_entropy(df["class"])
    x = w_avg * entropy
    info += x
  ig = find_entropy(data["class"]) - info
  return round(ig, 3) 

def entropy_and_infogain(data, feature):
  for i in range(train[feature].nunique()):
    df = data[data[feature]==train[feature].unique()[i]]
    if df.shape[0] < 1:
        continue
    
    # display(df[[feature, 'class']].style.applymap(highlight)\
    #         .set_properties(subset=[feature, 'class'], **{'width': '80px'})\
    #         .set_table_styles([{'selector': 'th', 'props': [('background-color', 'lightgray'), 
    #                                                         ('border', '1px solid gray'), 
    #                                                         ('font-weight', 'bold')]},
    #                             {'selector': 'td', 'props': [('border', '1px solid gray')]},
    #                             {'selector': 'tr:hover', 'props': [('background-color', 'white'), 
    #                                                               ('border', '1.5px solid black')]}]))
    
    print(f'Entropy of {feature} - {train[feature].unique()[i]} = {find_entropy(df["class"])}')
  print(f'Information Gain for {feature} = {information_gain(data, data[feature])}')

In [164]:
print(f'Entropy of the entire dataset: {find_entropy(train["class"])}')

Entropy of the entire dataset: 0.999


In [165]:
entropy_and_infogain(train, 'class')

Entropy of class - p = 0.0
Entropy of class - e = 0.0
Information Gain for class = 0.999


In [166]:
mushroom_raw_data[mushroom_raw_data["class"] == "p"]["class"].unique()[0]

'p'

In [167]:
train.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [168]:
def ordered_info_gain(data):
  info_gain_list = []
  attributes = data.columns
  for i in range(1, len(attributes)):
    info_gain_list.append([attributes[i],information_gain(data, data[attributes[i]])])
  info_gain_list.sort(reverse = True, key = lambda tuple: tuple[1],)
  return info_gain_list

In [333]:
def make_sub_tree(root, data_set):
  attribute_options = data_set[root].unique()
  tree = {root: {}}

  for current_option in attribute_options:
    tree[root].update({current_option: {}})
    current_data_set = data_set[data_set[root] == current_option].copy()
    current_class_options = data_set[data_set[root] == current_option]["class"]

    if current_class_options.nunique() > 1:
      next_attribute = ordered_info_gain(current_data_set)[0][0]
      tree[root][current_option].update(make_sub_tree(next_attribute, current_data_set))
      continue

    tree[root][current_option] = current_class_options.unique()[0]
    current_data_set.drop(root, inplace=True, axis=1)

  return tree

In [370]:
import json

def train_tree(data):
  data_copy = data.copy()
  info_gain_list = ordered_info_gain(data_copy)

  final_tree = make_sub_tree(info_gain_list[0][0], data_copy)
  print(json.dumps(final_tree, indent=4))
  return final_tree

In [371]:
tree = train_tree(train)

{
    "odor": {
        "p": "p",
        "l": "e",
        "f": "p",
        "y": "p",
        "s": "p",
        "n": {
            "spore-print-color": {
                "k": "e",
                "n": "e",
                "w": {
                    "habitat": {
                        "g": "e",
                        "l": {
                            "cap-color": {
                                "c": "e",
                                "y": "p",
                                "n": "e",
                                "w": "p"
                            }
                        },
                        "w": "e",
                        "d": {
                            "cap-surface": {
                                "y": "p",
                                "f": "p",
                                "s": "e"
                            }
                        },
                        "p": "e"
                    }
                },
                "r": "p",
            

In [400]:
def predict(model, data, attribute = ""):
  if isinstance(model, str):
        return model
  elif isinstance(model, dict):
    model_keys = list(model.keys())
    if len(model_keys) == 1:
      prediction = predict(model[model_keys[0]], data, model_keys[0])
    else:
      data_attribute = data[attribute]
      prediction = predict(model[data_attribute], data)
    return prediction


In [403]:
predict(tree, test.iloc[0])

'e'

In [404]:
test.iloc[0]["class"]

'e'

In [445]:
def evaluate_prediction(data_set, model, target):
  correct = 0
  wrong = 0
  wrong_indexes = []
  for i in range(len(data_set)):
    prediction = predict(model, data_set.iloc[i])
    if prediction == data_set.iloc[i][target]: 
      correct += 1
    else: 
      # print(prediction, data_set.iloc[i].index)
      wrong += 1
      wrong_indexes.append(i)

  result = (correct / (correct + wrong)) * 100
  return correct, wrong, result, wrong_indexes

In [446]:
evaluate_prediction(test, tree, "class")

(2434, 4, 99.83593109105826, [253, 715, 966, 1795])

In [452]:
print(test.iloc[253],'\n')
print(test.iloc[715],'\n')
print(test.iloc[966],'\n')
print(test.iloc[1795],'\n')

class                       e
cap-shape                   f
cap-surface                 y
cap-color                   n
bruises                     f
odor                        n
gill-attachment             f
gill-spacing                c
gill-size                   b
gill-color                  w
stalk-shape                 e
stalk-root                  b
stalk-surface-above-ring    y
stalk-surface-below-ring    y
stalk-color-above-ring      n
stalk-color-below-ring      n
veil-type                   p
veil-color                  w
ring-number                 t
ring-type                   p
spore-print-color           w
population                  y
habitat                     d
Name: 7292, dtype: object 

class                       e
cap-shape                   b
cap-surface                 y
cap-color                   n
bruises                     f
odor                        n
gill-attachment             f
gill-spacing                c
gill-size                   b
gill-color  